In [23]:
# import library
import pandas as pd
import numpy as np 
import matlab 
import matplotlib as plt
import os

In [55]:
# read the file
# mortality
mortality_path = r'data\Morticd10_part1.csv'
mortality_data_2002 = pd.read_csv(mortality_path, low_memory=False)
print(mortality_data_2002.head()) 
print(mortality_data_2002.shape)
    

   Country  Admin1  SubDiv  Year List Cause  Sex  Frmat  IM_Frmat  Deaths1   
0     1400     NaN     NaN  2001  101  1000    1      7         8      332  \
1     1400     NaN     NaN  2001  101  1000    2      7         8      222   
2     1400     NaN     NaN  2001  101  1001    1      7         8       24   
3     1400     NaN     NaN  2001  101  1001    2      7         8       14   
4     1400     NaN     NaN  2001  101  1002    1      7         8        0   

   ...  Deaths21  Deaths22  Deaths23  Deaths24  Deaths25  Deaths26   
0  ...      95.0       NaN       NaN       NaN       NaN       0.0  \
1  ...     112.0       NaN       NaN       NaN       NaN       0.0   
2  ...       5.0       NaN       NaN       NaN       NaN       0.0   
3  ...       6.0       NaN       NaN       NaN       NaN       0.0   
4  ...       0.0       NaN       NaN       NaN       NaN       0.0   

   IM_Deaths1  IM_Deaths2  IM_Deaths3  IM_Deaths4  
0         8.0         NaN         NaN         NaN  
1     

In [25]:
# population
pop_path = r'data\pop.csv'
pop_data = pd.read_csv(pop_path, low_memory=False)

In [ ]:
# country code
country_code_path = r'data\country_codes.csv'
country_code = pd.read_csv(country_code_path, low_memory=False)

43    Seychelles
Name: name, dtype: object